In [16]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import datetime
import pickle

In [17]:
checkpoint_filepath = 'tmp/checkpoint'

model = keras.models.load_model(f"{checkpoint_filepath}/01-0.80")

2023-01-08 12:10:56.780545: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-01-08 12:10:56.810215: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-01-08 12:10:57.644474: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-01-08 12:10:57.672685: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-01-08 12:10:58.706168: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond/while' has 13 outputs but the _output_sh

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 64)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 64, 64)            5120000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              66048     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0

In [27]:
df = pd.read_csv("datasets/test2.csv", encoding = "ISO-8859-1")
df = df.sample(frac=1)[:] # shuffle and truncate
print(len(df.loc[df['sentiment']=="Neutral"]))
print(len(df))
df.drop(df.loc[df['sentiment']=="Neutral"].index, inplace=True)
df['sentiment'] = df['sentiment'].apply(lambda x: 1 if x == "Positive" else 0)

print(len(df))

3142
13871
10729


In [64]:
import sklearn

print(model.predict(np.array(["Shit!", "Oh fuck!", "You'd better shut up!", "I wanna kill this bastard", "That was amazing", "OMG!"
                        "That was cute", "Nice one", "I loved it", "I really liked how he behaved", "He was a nice dude"])))


df = pd.read_csv("datasets/test.csv", encoding = "ISO-8859-1", names=['polarity', 'id', 'query', 'user', 'text'], index_col=2)
df = df.sample(frac=1)[:] # shuffle and truncate
df.drop(df.loc[df['polarity']==2].index, inplace=True)
print(set(df['polarity'].values))

map_val = {0:0, 4:2, 2:1}

real_labels = df['polarity'].map(lambda x: map_val[x])
df['polarity'] = df['polarity'].apply(lambda x: 1 if x == 4 else 0)

test_data = df['text'].to_numpy()
test_label = df['polarity'].to_numpy()

def map_vals(x):
    if x < 0.5:
        return 0
    elif x > 0.5:
        return 2
    return 1

predictions = (model.predict(test_data))
predictions = [map_vals(x) for x in predictions]

print(list(predictions - real_labels).count(0) / len(predictions))
print(sklearn.metrics.confusion_matrix(real_labels, predictions))

print(model.evaluate(test_data, test_label))


df = pd.read_csv("datasets/test2.csv", encoding = "ISO-8859-1")
df = df.sample(frac=1)[:] # shuffle and truncate
df.drop(df.loc[df['sentiment']=="Neutral"].index, inplace=True)
print(set(df['sentiment'].values))
df['sentiment'] = df['sentiment'].apply(lambda x: 1 if x == "Positive" else 0)


test_data = df['text'].to_numpy()
test_label = df['sentiment'].to_numpy()

print(model.evaluate(test_data, test_label))

1/1 [==============================] - 0s 49ms/step
[[-1.536935 ]
 [-1.6616406]
 [ 1.6750636]
 [-3.091279 ]
 [ 3.0937948]
 [ 1.9968209]
 [ 2.7224426]
 [ 2.3869634]
 [ 1.6188831]
 [ 1.978704 ]]
{0, 4}
12/12 [==============================] - 0s 35ms/step
0.8217270194986073
[[154  23]
 [ 41 141]]
12/12 [==============================] - 0s 35ms/step - loss: 0.4073 - accuracy: 0.8217
[0.40732038021087646, 0.8217270374298096]
{'Positive', 'Negative'}
336/336 [==============================] - 11s 32ms/step - loss: 1.0349 - accuracy: 0.5240
[1.0348516702651978, 0.524000346660614]
